In [1]:
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

import json
import pandas as pd
import os
import numpy as np

In [2]:
# Set environment variables
PLACES_KEY =  os.getenv('PLACES_API_KEY')

In [3]:
# Set service
service = build('places', 'v1', developerKey=PLACES_KEY)

In [4]:
def places_connection(api_key):
    try:
        # Connect to Google Places API
        service = build('places', 'v1', developerKey=api_key)        
               
        return service
    
    except HttpError as error:
        print(f'An error occurred: {error}')
        return None

In [5]:
service = places_connection(PLACES_KEY)

In [9]:
# Load cities
cities = pd.read_csv('tests\cities.csv')

In [10]:
cities[:3]

,city
0,Altamonte-Springs
1,Apopka
2,Bartow


## PLACES TO STAY

In [12]:
places_to_stay = ['hotel', 'inn', 'resort_hotel','bed_and_breackfast'] # List to store places to stay

In [13]:
# Loop through each city
def get_places_to_stay(cities, places_to_stay):
    stay_list = []  # List to store places to stay

    for city in cities:

        for place in places_to_stay:

            try:
                # Prepare the initial search query
                query = f'{place} in {city}, Florida, USA'
                request_body = {
                    'textQuery': query,
                    'regionCode': 'US',
                    'languageCode':'en',
                    'includedType': 'cell_phone_store',
                    'pageSize': 5
                }
                
                # Make the request to the   
                response = service.places().searchText(body=request_body, fields='*').execute()
                
                # Process the current page of results
                if 'places' in response:
                    for place in response['places']:
                        store_data = {
                            'City': city,
                            'Store Name': place.get('displayName', {}).get('text', 'N/A'),
                            'Address': place.get('formattedAddress', 'N/A'),
                            'Latitude': place.get('location', {}).get('latitude', np.nan),
                            'Longitude': place.get('location', {}).get('longitude', np.nan),
                            'Rating': place.get('rating', np.nan),
                            'User Rating Count': place.get('userRatingCount', 0)
                        }
                        stay_list.append(store_data)                    
                        
            except Exception as e:
                    print(f"Error processing {city}: {e}")                
                    continue

            # Convert to DataFrames
        df = pd.DataFrame(stay_list)

    return df

In [15]:
df_stay = get_places_to_stay(cities['city'], places_to_stay)

In [16]:
df_stay.head()

,City,Store Name,Address,Latitude,Longitude,Rating,User Rating Count
0,Altamonte-Springs,Hampton Inn & Suites Orlando-North/Altamonte S...,"161 Douglas Ave, Altamonte Springs, FL 32714",28.664357,-81.389789,4.4,1062
1,Altamonte-Springs,Altamonte Springs Hotel and Suites,"450 Douglas Ave #302, Altamonte Springs, FL 32714",28.668465,-81.390878,2.0,314
2,Altamonte-Springs,Hilton Orlando/Altamonte Springs,"350 Northlake Blvd, Altamonte Springs, FL 32701",28.654494,-81.387308,4.2,2202
3,Altamonte-Springs,Quality Inn & Suites,"150 S Westmonte Dr, Altamonte Springs, FL 32714",28.660765,-81.394026,2.9,1154
4,Altamonte-Springs,Ramada by Wyndham Altamonte Springs Near I-4,"150 Douglas Ave, Altamonte Springs, FL 32714",28.663300,-81.390692,3.3,879


In [17]:
df_stay.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1066 entries, 0 to 1065
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   City               1066 non-null   object 
 1   Store Name         1066 non-null   object 
 2   Address            1066 non-null   object 
 3   Latitude           1066 non-null   float64
 4   Longitude          1066 non-null   float64
 5   Rating             1060 non-null   float64
 6   User Rating Count  1066 non-null   int64  
dtypes: float64(3), int64(1), object(3)
memory usage: 58.4+ KB


In [33]:
# Filtering the data    
filtered_df = df_stay[(df_stay['Rating'] > 4) & (df_stay['User Rating Count'] > 100)]

In [35]:
filtered_df = filtered_df.drop_duplicates(subset='Latitude')

In [36]:
filtered_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 167 entries, 0 to 1029
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   City               167 non-null    object 
 1   Store Name         167 non-null    object 
 2   Address            167 non-null    object 
 3   Latitude           167 non-null    float64
 4   Longitude          167 non-null    float64
 5   Rating             167 non-null    float64
 6   User Rating Count  167 non-null    int64  
dtypes: float64(3), int64(1), object(3)
memory usage: 10.4+ KB


In [37]:
filtered_df.sample(10)

,City,Store Name,Address,Latitude,Longitude,Rating,User Rating Count
80,Bartow,Hampton Inn Bartow,"205 Old Bartow Eagle Lake Rd, Bartow, FL 33830",27.908513,-81.841798,4.5,775
484,Lakeland,"Holiday Inn Express & Suites Lakeland South, a...","3405 Florida Ave S, Lakeland, FL 33803",28.002885,-81.958122,4.5,403
161,Clermont,Hampton Inn & Suites Clermont,"2200 State Rte 50, Clermont, FL 34711",28.547627,-81.714935,4.4,651
574,Ocala,"Studio 6 Ocala, FL","3620 W Silver Springs Blvd, Ocala, FL 34475",29.184814,-82.182388,4.4,279
297,Kissimmee,Quality Inn & Suites By the Parks,"2945 Entry Point Blvd, Kissimmee, FL 34747",28.334167,-81.592222,4.1,1308
402,Lake-Placid,Lake Grassy Inn & Suites,"1865 US-27, Lake Placid, FL 33852",27.253669,-81.340078,4.4,126
218,Dunnellon,Retreat at Crystal Manatee,"310 N Citrus Ave, Crystal River, FL 34428",28.897319,-82.593357,4.4,432
364,Lake-Mary,Holiday Inn Express & Suites Sanford- Lake Mar...,"4750 FL-46, Sanford, FL 32771",28.812570,-81.336250,4.5,347
570,Ocala,Shamrock Cove Inn,"215 S Pine Ave, Ocala, FL 34471",29.184350,-82.140078,4.4,350
761,St-Cloud,Budget Inn of St. Cloud,"602 13th St, St Cloud, FL 34769",28.246266,-81.275998,4.5,294


## Nodes PLACE_TO_STAY

In [38]:
filtered_df.rename(columns={'Store Name': 'Place_to_Stay'}, inplace=True)

In [39]:
filtered_df.sample(10)

,City,Place_to_Stay,Address,Latitude,Longitude,Rating,User Rating Count
745,Sebring,Buttonwood Bay,"10001 US-27, Sebring, FL 33876",27.407385,-81.417174,4.5,333
955,Winter-Garden,Orange County National Golf Center and Lodge,"16301 Phil Ritson Way, Winter Garden, FL 34787",28.443380,-81.629026,4.4,1868
2,Altamonte-Springs,Hilton Orlando/Altamonte Springs,"350 Northlake Blvd, Altamonte Springs, FL 32701",28.654494,-81.387308,4.2,2202
307,Kissimmee,FantasyWorld Resort,"5005 Kyngs Heath Rd, Kissimmee, FL 34746",28.335294,-81.485288,4.3,3497
7,Altamonte-Springs,Sheraton Orlando North Hotel,"600 N Lake Destiny Rd, Maitland, FL 32751",28.632097,-81.391331,4.1,2824
441,Lake-Wales,"Holiday Inn Express Lake Wales N-Winter Haven,...","2953 Ridge Way, Lake Wales, FL 33859",27.962103,-81.622583,4.4,685
648,Oviedo,Fairfield Inn & Suites Orlando East/UCF Area,"3420 Lake Lynda Dr, Orlando, FL 32817",28.598872,-81.222999,4.6,728
991,Winter-Haven,Courtyard Winter Haven,"6225 Cypress Gardens Blvd, Winter Haven, FL 33884",27.985427,-81.684697,4.5,603
527,Mount-Dora,Microtel Inn & Suites by Wyndham Leesburg/Mt Dora,"9700 US-441, Leesburg, FL 34788",28.825037,-81.795911,4.1,770
149,Celebration,Westgate Vacation Villas Resort,"7700 Westgate Blvd, Kissimmee, FL 34747",28.328955,-81.593123,4.2,15025


In [41]:
filtered_df.to_csv(r'neo4j\nodes\node_places_to_stay.csv', index=False, encoding='utf-8')

In [42]:
r_places_to_stay = filtered_df[['Place_to_Stay', 'City']].copy()

In [43]:
r_places_to_stay['relation'] = 'PLACE_TO_STAY'

In [44]:
r_places_to_stay.head(5)

,Place_to_Stay,City,relation
0,Hampton Inn & Suites Orlando-North/Altamonte S...,Altamonte-Springs,PLACE_TO_STAY
2,Hilton Orlando/Altamonte Springs,Altamonte-Springs,PLACE_TO_STAY
5,SpringHill Suites Orlando Altamonte Springs/Ma...,Altamonte-Springs,PLACE_TO_STAY
7,Sheraton Orlando North Hotel,Altamonte-Springs,PLACE_TO_STAY
28,TownePlace Suites Orlando Altamonte Springs/Ma...,Altamonte-Springs,PLACE_TO_STAY


In [46]:
r_places_to_stay.to_csv(r'neo4j\relationships\relation_places_to_stay.csv', index=False, encoding='utf-8')